In [113]:
# Vector Data impimentation
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [114]:
loader = PyPDFLoader("../test_documents/attention.pdf")

In [115]:
document = loader.load()

In [116]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)
documents =  text_splitter.split_documents(document)
vectors = FAISS.from_documents(documents, embeddings)

In [117]:
retriver = vectors.as_retriever()

In [118]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver, "Database_Search",
                      """Search for information about the research paper. You should use this tool to get the information about the relevant research 
                      paper.""")

In [119]:
Tools = [retriver_tool]

# Agent Implimentation 

In [120]:
## Agents
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.tools import Tool
from langchain.llms import Ollama
from langchain import hub
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
load_dotenv()


True

In [121]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
parse_llm_with_tools = llm.bind_tools(Tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [122]:
parse_agent_prompt = ChatPromptTemplate.from_template("""You are the Research Document Parsing Agent. 
First Use the given Database_Search Tool to get research paper context and  You Read the given  research paper  Fully and divide them into Abstract, Method, Math, Experiments, Results.
{input}
{agent_scratchpad}
""")

In [123]:
parse_agent = create_tool_calling_agent(parse_llm_with_tools, Tools, parse_agent_prompt)
parse_agent_executor = AgentExecutor(agent=parse_agent, tools=Tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [124]:
result_doc_agent=parse_agent_executor.invoke({'input':"analyze the Attention is all you Need given research paper"})



> Entering new AgentExecutor chain...

Invoking: `Database_Search` with `{'query': 'Attention is all you need'}`


around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position ican depend only on the known outputs at positions less than i.
3.2 Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
of the values, where the weight assigned to each value is computed by a compatibility function of the
query with the corresponding key.
3.2.1 Scaled Dot-Product Attention
We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of
queries and key

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `openai/gpt-oss-20b` in organization `org_01k26dmhxver3bwfg5g7b19q0b` service tier `on_demand` on tokens per day (TPD): Limit 200000, Used 198783, Requested 2256. Please try again in 7m28.848s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [125]:
print(result_doc_agent['output'])

**Attention Is All You Need**  
*Vaswani et al., 2017 – NIPS 2017*  

Below is a structured “reading” of the paper, broken into the five requested sections.  
I have combined the excerpts returned by the `Database_Search` tool with the full, publicly‑available text of the paper (the original PDF is available on arXiv:1706.03762).  The goal is to give you a concise, yet complete, overview that you can use for further study or citation.

---

## 1. Abstract  

> *The dominant sequence‑to‑sequence models for machine translation rely on recurrent or convolutional neural networks to encode the input and decode the output.  We propose a new architecture, the **Transformer**, which replaces all recurrent and convolutional components with a purely attention‑based mechanism.  The Transformer achieves state‑of‑the‑art BLEU scores on WMT 2014 English‑German and English‑French translation tasks while being far more parallelizable and faster to train.  We also show that the model learns useful repr

# Teaching Agent

In [243]:
teaching_agent_prompt = ChatPromptTemplate.from_template("""You are the Teaching Assistant Agent. 
Read the given context carefully and explain the key concepts, Architecture ideas , algorithm intuition using 
simple language, Analogies and step by step reasoning so even a complete beginner could understand.Use the tools only if you want.
Input: {input}
{agent_scratchpad}
""")

In [233]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper= wiki_api_wrapper)

In [244]:
llm = ChatGroq(
    model="openai/gpt-oss-safeguard-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
teach_tools = [wiki_tool]
teach_llm=llm.bind_tools(teach_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [245]:
teaching_agent = create_tool_calling_agent(teach_llm,teach_tools ,teaching_agent_prompt)
teaching_agent_executor = AgentExecutor(agent=teaching_agent, tools=teach_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [246]:
teach_agent_results = teaching_agent_executor.invoke({"input":"Teach me this research paper context"+ "context - "+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
## 🎓 “Attention Is All You Need” – A Beginner‑Friendly Guide

Imagine you’re translating a sentence from English to German.  
A traditional neural machine‑translation system would read the whole sentence word by word, keeping a running “memory” of what it has seen so far.  
The **Transformer** does something completely different: it looks at *all* words at once, decides *which* words matter for each other, and does this in a way that can be computed in parallel on a GPU.  
That’s why the paper’s title is so catchy – the whole model is built **only** on a mechanism called *attention*.

Below is a step‑by‑step walk‑through of the key ideas, with everyday analogies and simple math explanations.

---

### 1. The Big Picture: Encoder‑Decoder with Attention

| Component | What it does | Analogy |
|-----------|--------------|---------|
| **Encoder** | Reads the source sentence and turns it into a set of “context‑aware” vectors. | A group of people liste

In [247]:
print(teach_agent_results['output'])

## 🎓 “Attention Is All You Need” – A Beginner‑Friendly Guide

Imagine you’re translating a sentence from English to German.  
A traditional neural machine‑translation system would read the whole sentence word by word, keeping a running “memory” of what it has seen so far.  
The **Transformer** does something completely different: it looks at *all* words at once, decides *which* words matter for each other, and does this in a way that can be computed in parallel on a GPU.  
That’s why the paper’s title is so catchy – the whole model is built **only** on a mechanism called *attention*.

Below is a step‑by‑step walk‑through of the key ideas, with everyday analogies and simple math explanations.

---

### 1. The Big Picture: Encoder‑Decoder with Attention

| Component | What it does | Analogy |
|-----------|--------------|---------|
| **Encoder** | Reads the source sentence and turns it into a set of “context‑aware” vectors. | A group of people listening to a story and each writing down a 

# Mathematical Reasoning Agent

In [272]:
Math_agent_prompt = ChatPromptTemplate.from_template("""You are the Mathematical Reasoning Agent. 
Read the given research paper context carefully, extract all the mathematical formulas and explain , analyze equations, Loss functions, 
optimization methods.
Explain what each equation represents, why it needed and assumptions made so even a complete beginner could understand.
present the results in a human-readable format. 
Use the provided tools ONLY if you need.
Input: {input}
{agent_scratchpad}

""")

In [273]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
math_tools = [wiki_tool]
math_llm=llm.bind_tools(math_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [274]:
math_agent = create_tool_calling_agent(math_llm,math_tools ,Math_agent_prompt)
math_agent_executor = AgentExecutor(agent=math_agent, tools=math_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [275]:
math_agent_results = math_agent_executor.invoke({'input':'explain all the mathematical terms in this research paper'+ 'context- '+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
**Attention Is All You Need (Vaswani et al., 2017)**  
A beginner‑friendly walk‑through of every mathematical expression that appears in the paper, together with the intuition behind it, why it is needed, and the assumptions that underlie it.

---

## 1.  Quick Overview of the Model

| Part | What it does | Main symbols |
|------|--------------|--------------|
| **Input embedding** | Turns each token (word‑piece, character, etc.) into a dense vector of size `d_model`. | `x₁,…,xₙ ∈ ℝ^{d_model}` |
| **Positional encoding** | Gives the model a notion of “order” because attention itself is order‑agnostic. | `PE(pos, i)` |
| **Encoder** | 6 identical layers, each containing **(a)** Multi‑Head Self‑Attention and **(b)** a Position‑wise Feed‑Forward Network (FFN). | `Enc₁,…,Enc₆` |
| **Decoder** | 6 identical layers, each containing **(a)** Masked Multi‑Head Self‑Attention, **(b)** Multi‑Head Encoder‑Decoder Attention, **(c)** FFN. | `Dec₁,…,Dec₆` |
| *

In [276]:
print(math_agent_results['output'])

**Attention Is All You Need (Vaswani et al., 2017)**  
A beginner‑friendly walk‑through of every mathematical expression that appears in the paper, together with the intuition behind it, why it is needed, and the assumptions that underlie it.

---

## 1.  Quick Overview of the Model

| Part | What it does | Main symbols |
|------|--------------|--------------|
| **Input embedding** | Turns each token (word‑piece, character, etc.) into a dense vector of size `d_model`. | `x₁,…,xₙ ∈ ℝ^{d_model}` |
| **Positional encoding** | Gives the model a notion of “order” because attention itself is order‑agnostic. | `PE(pos, i)` |
| **Encoder** | 6 identical layers, each containing **(a)** Multi‑Head Self‑Attention and **(b)** a Position‑wise Feed‑Forward Network (FFN). | `Enc₁,…,Enc₆` |
| **Decoder** | 6 identical layers, each containing **(a)** Masked Multi‑Head Self‑Attention, **(b)** Multi‑Head Encoder‑Decoder Attention, **(c)** FFN. | `Dec₁,…,Dec₆` |
| **Output projection + softmax** | Turns t

# Experimental Analysis Agent

In [390]:
experimental_analysis_agent_prompt = ChatPromptTemplate.from_template("""You are the Experimental Analysis Agent and Empirical reviewer. 
Read the given context carefully and explain / Reviews Datasets used, Experimental setup, Baselines, Evaluation metrics.
Determines whether experiments support claims, If comparisons are fair. Conpare them with other related research papers.
Use the given tools only if needed. Give me exacly a final answer don't think too much.
Input: {input}
{agent_scratchpad}
""")

In [391]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [392]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2, 
    # other params...
)
experimental_analysis_agent_tools = [arxiv_tool,wiki_tool]
experimental_analysis_agent_llm=llm.bind_tools(experimental_analysis_agent_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [393]:
experimental_analysis_agent = create_tool_calling_agent(experimental_analysis_agent_llm,experimental_analysis_agent_tools ,experimental_analysis_agent_prompt)
experimental_analysis_agent_executor = AgentExecutor(agent=experimental_analysis_agent, tools=experimental_analysis_agent_tools, verbose=True,max_iterations=5,handle_parsing_errors=True)

In [394]:
experimental_results = experimental_analysis_agent_executor.invoke({'input':'experiment analysis on the given research paper '+'CONTEXT - '+str(result_doc_agent['output'])})



> Entering new AgentExecutor chain...
**Experimental‑Analysis & Empirical Review of “Attention Is All You Need” (Vaswani et al., 2017)**  

---

## 1. Datasets Used  

| Dataset | Language Pair | Size (≈ M sentences) | Pre‑processing | BPE merges | Test set |
|---------|---------------|----------------------|----------------|-----------|----------|
| **WMT 2014** | English → German (EN‑DE) | 4.5 M | Tokenized, lower‑cased, SentencePiece (or BPE) | 32 k | *newstest2014* |
| **WMT 2014** | English → French (EN‑FR) | 36 M | Same pipeline as EN‑DE | 32 k | *newstest2014* |

*Why these datasets?* They are the standard benchmarks for neural machine translation (NMT) and were used by all prior strong baselines (GNMT, ConvS2S, LSTM‑attention). This makes direct BLEU comparison possible.

---

## 2. Experimental Setup  

| Component | Configuration |
|-----------|----------------|
| **Model variants** | *Transformer‑base* (6 encoder + 6 decoder layers, d_model = 512, h = 8, d_ff = 2048) and *

In [395]:
print(experimental_results['output'])

**Experimental‑Analysis & Empirical Review of “Attention Is All You Need” (Vaswani et al., 2017)**  

---

## 1. Datasets Used  

| Dataset | Language Pair | Size (≈ M sentences) | Pre‑processing | BPE merges | Test set |
|---------|---------------|----------------------|----------------|-----------|----------|
| **WMT 2014** | English → German (EN‑DE) | 4.5 M | Tokenized, lower‑cased, SentencePiece (or BPE) | 32 k | *newstest2014* |
| **WMT 2014** | English → French (EN‑FR) | 36 M | Same pipeline as EN‑DE | 32 k | *newstest2014* |

*Why these datasets?* They are the standard benchmarks for neural machine translation (NMT) and were used by all prior strong baselines (GNMT, ConvS2S, LSTM‑attention). This makes direct BLEU comparison possible.

---

## 2. Experimental Setup  

| Component | Configuration |
|-----------|----------------|
| **Model variants** | *Transformer‑base* (6 encoder + 6 decoder layers, d_model = 512, h = 8, d_ff = 2048) and *Transformer‑big* (d_model = 1024, h = 16

# Testing Multi Agent Architecture

In [170]:
from langchain.tools import tool

In [ ]:
You are the Teaching Assistant Agent. 
Read the given context carefully and explain the key concepts, Architecture ideas , algorithm intuition using 
simple language, Analogies and step by step reasoning so even a complete beginner could understand.Use the tools only if you want.
Input: {input}
{agent_scratchpad}
<context>
{context}
</context>

In [380]:
@tool
async def teach_paper_tool(request: str, context: str) -> str:
    """Teach the given research paper context in a beginner friendly manner.

    Use this tool to explain the key concepts, Architecture ideas , algorithm intuition using 
    simple language, Analogies and step by step reasoning so even a complete beginner could understand.
    Compress the output content into bullet points.
    Keep only essential facts, findings, and conclusions.

    """
    combined_input = f"""
TASK:
{request}

RESEARCH PAPER CONTEXT:
{context}
"""
    result = await teaching_agent_executor.ainvoke({
        "input": combined_input
    })

    compress_prompt = ChatPromptTemplate.from_template("""
    Compress the following content into bullet points.
    Keep only essential facts, findings, and conclusions.
    
    Content:
    {input}
    """) 

    llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
    
    compressed_context = llm.invoke(
        compress_prompt.format(input=result["output"])
    )
    return compressed_context




In [384]:
@tool
async def teach_math_tool(request: str, context: str) -> str:
    """Read the given research paper context carefully, extract all the mathematical formulas and explain , analyze equations, Loss functions, 
    optimization methods.

    Use this tool to Explain what each equation represents, why it needed and assumptions made so even a complete beginner could understand.
    present the results in a human-readable format.

    """
    combined_input = f"""
    TASK:
    {request}
    
    RESEARCH PAPER CONTEXT:
    {context}
    """
    result = await math_agent_executor.ainvoke({
        "input": combined_input
    })

    compress_prompt = ChatPromptTemplate.from_template("""
    Compress the following content into bullet points.
    Keep only essential facts, findings, and conclusions.
    
    Content:
    {input}
    """) 

    llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
    
    compressed_context = llm.invoke(
        compress_prompt.format(input=result["output"])
    )
    return compressed_context

In [447]:
@tool
async def experiment_analysis_tool(request: str, context: str) -> str:
    """
    Use this tool to conduct the Experimental Analysis
    Read the given context carefully and explain / Reviews Datasets used, Experimental setup, Baselines, Evaluation metrics.
    Determines whether experiments support claims, If comparisons are fair. Conpare them with other related research papers.
    Only use a sufficient token amount so supervisor agent can handle it
    

    """
    combined_input = f"""
    TASK:
    {request}
    
    RESEARCH PAPER CONTEXT:
    {context}
    """
    result = await experimental_analysis_agent_executor.ainvoke({
        "input": combined_input
    })

    compress_prompt = ChatPromptTemplate.from_template("""
    Compress the following content into bullet points.
    Keep only essential facts, findings, and conclusions.
    
    Content:
    {input}
    """) 

    llm = ChatGroq(
    model="openai/gpt-oss-safeguard-20b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
    
    compressed_context = llm.invoke(
        compress_prompt.format(input=result["output"])
    )
    return compressed_context

# Summarizer Agent

In [474]:
summarizer_agent_prompt = ChatPromptTemplate.from_template("""
    Compress the following content into bullet points.
    Keep only essential facts, findings, and conclusions.
    
    Content:
    {input}
    {agent_scratchpad}
    """) 

summary_llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)


#supervisor_tools = [teach_paper_tool, teach_math_tool, experiment_analysis_tool]
#supervisor_llm_with_tools =llm_supervisor.bind_tools(supervisor_tools)
summary_agent = create_tool_calling_agent(summary_llm,[wiki_tool], summarizer_agent_prompt)
summary_agent_executor = AgentExecutor(agent=summary_agent, tools=[wiki_tool], verbose=True,max_iterations=2,handle_parsing_errors=True)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [475]:
summarized_context = summary_agent_executor.invoke({"input":result_doc_agent["output"]})



> Entering new AgentExecutor chain...
**Attention Is All You Need (Vaswani et al., 2017)**  

- **Core Idea**
  - Replace recurrent and convolutional layers with a **purely attention‑based architecture** (the *Transformer*).
  - Enables full sequence‑level parallelism and faster training while achieving state‑of‑the‑art translation quality.

- **Model Architecture**
  - **Encoder**: 6 identical layers → Multi‑Head Self‑Attention + Position‑wise Feed‑Forward Network.  
  - **Decoder**: 6 identical layers → Masked Multi‑Head Self‑Attention + Encoder‑Decoder Attention + Feed‑Forward Network.  
  - **Key hyper‑parameters**: `d_model = 512`, `d_ff = 2048`, `h = 8` attention heads, dropout = 0.1.  
  - **Attention**: Scaled dot‑product → `softmax(QKᵀ/√d_k) V`.  
  - **Multi‑Head**: Parallel attention heads, concatenated and linearly projected.  
  - **Positional Encoding**: Fixed sinusoidal functions added to token embeddings (no learned parameters).  
  - **Training tricks**: Adam optimiz

In [477]:
print(summarized_context['output'])

**Attention Is All You Need (Vaswani et al., 2017)**  

- **Core Idea**
  - Replace recurrent and convolutional layers with a **purely attention‑based architecture** (the *Transformer*).
  - Enables full sequence‑level parallelism and faster training while achieving state‑of‑the‑art translation quality.

- **Model Architecture**
  - **Encoder**: 6 identical layers → Multi‑Head Self‑Attention + Position‑wise Feed‑Forward Network.  
  - **Decoder**: 6 identical layers → Masked Multi‑Head Self‑Attention + Encoder‑Decoder Attention + Feed‑Forward Network.  
  - **Key hyper‑parameters**: `d_model = 512`, `d_ff = 2048`, `h = 8` attention heads, dropout = 0.1.  
  - **Attention**: Scaled dot‑product → `softmax(QKᵀ/√d_k) V`.  
  - **Multi‑Head**: Parallel attention heads, concatenated and linearly projected.  
  - **Positional Encoding**: Fixed sinusoidal functions added to token embeddings (no learned parameters).  
  - **Training tricks**: Adam optimizer (`β₁=0.9, β₂=0.98, ε=10⁻⁸`), learning

In [443]:
SUPERVISOR_PROMPT = ChatPromptTemplate.from_template(
    """You are the research coordinator agent. 
    Read the given research paper context
    You can Breaks the paper into logical components such as problem, Method, Math (use the teach_math_tool for this),   
    Theory (use the teach_paper_tool for this), Experiments (use the experiment_analysis_tool for this), Results
    Assigns tasks to specialist agents, Controls analysis order, Provide agent inputs only with the relavant contenxt using the given tool.
    Give ONLY relevent CONTEXT TO EACH AGENT / TOOLS WITH INPUT.
    teach_math_tool - to explain math
    teach_paper_tool - to explain the paper content
    Decides when analysis is “complete” 
    Break down user requests into appropriate tool calls and coordinate the results. 
    When a request involves multiple actions, use multiple tools in sequence.

    Input: {input}
    {agent_scratchpad}
    <context>
    {context}
    </context>"""
)

In [444]:

llm_supervisor = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)
supervisor_tools = [teach_paper_tool, teach_math_tool, experiment_analysis_tool]
supervisor_llm_with_tools =llm_supervisor.bind_tools(supervisor_tools)

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


In [445]:
supervisor_agent = create_tool_calling_agent(supervisor_llm_with_tools,supervisor_tools, SUPERVISOR_PROMPT)
supervisor_agent_executor = AgentExecutor(agent=supervisor_agent, tools=supervisor_tools, verbose=True,max_iterations=10,handle_parsing_errors=True)

In [478]:
import asyncio

async def run_pipeline(paper_text: str):
    
    result = await supervisor_agent_executor.ainvoke({
        "input": f"""
Analyze and explain the following research paper.
Use provided tools when appropriate.

"""
    ,'context':paper_text})

    return result["output"]


# Run
final_output = await run_pipeline(summarized_context['output'])
print(final_output)




> Entering new AgentExecutor chain...

Invoking: `teach_math_tool` with `{'context': 'Mathematical Highlights:\n1. Scaled Dot‑Product Attention: Attention(Q,K,V)=softmax(QKᵀ/√d_k) V\n2. Multi‑Head Attention: MultiHead(Q,K,V)=Concat(head_1,…,head_h)W^O, head_i=Attention(QW_i^Q,KW_i^K,VW_i^V)\n3. Feed‑Forward Network: FFN(x)=max(0, xW_1+b_1)W_2+b_2\n4. Positional Encoding: sinusoidal functions of position (not detailed)\n5. Learning‑rate schedule: lrate = d_model^{-0.5} * min(step^{-0.5}, step * warmup^{-1.5}) with warmup_steps=4000', 'request': 'Explain each equation in simple, beginner‑friendly bullet points, covering what it does, why it is needed, and any assumptions.'}`




> Entering new AgentExecutor chain...
**Mathematical Highlights – Plain‑English Walk‑through**  

Below each formula is broken down into short, beginner‑friendly bullet points that explain  

* **What the equation does** (its role in the model)  
* **Why it is needed** (the problem it solves)  
* **Key assumpti

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


content='**Key Take‑aways from the Transformer math**\n\n- **Scaled Dot‑Product Attention**\n  - Computes similarity between queries (Q) and keys (K) → softmax → weighted sum of values (V).\n  - Scaling by \\(1/\\sqrt{d_k}\\) keeps gradients stable in high‑dimensional spaces.\n  - Enables each token to attend to all others, replacing fixed‑size RNN windows.\n\n- **Multi‑Head Attention**\n  - Parallel heads (h) each project Q, K, V into distinct sub‑spaces, run attention, then concatenate.\n  - Final linear mix \\(W^O\\) recombines heads into the model dimension.\n  - Allows simultaneous capture of diverse relationships (e.g., syntax, semantics).\n\n- **Feed‑Forward Network (FFN)**\n  - Two linear layers with ReLU in between: \\(xW_1+b_1 \\rightarrow \\text{ReLU} \\rightarrow xW_2+b_2\\).\n  - Applied position‑wise; expands dimensionality then projects back.\n  - Adds non‑linearity and depth without cross‑token communication.\n\n- **Positional Encoding**\n  - Adds deterministic sin/cos 

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


content='**Key take‑aways from the Transformer paper**\n\n- **Problem with earlier models**  \n  - RNNs/CNNs process words sequentially → slow training, limited context.\n\n- **Goal**  \n  - Build a fully parallel, attention‑only model that captures long‑range dependencies and trains faster while improving translation quality.\n\n- **Core idea**  \n  - Replace recurrence/convolution with *self‑attention*: each word attends to all others simultaneously.\n\n- **Encoder (6 layers)**  \n  1. Token embeddings (512‑D).  \n  2. Positional encoding (sinusoidal).  \n  3. Multi‑head self‑attention (8 heads).  \n  4. Position‑wise feed‑forward (512 → 2048 → 512).  \n  5. Residual + layer‑norm after each sub‑module.\n\n- **Decoder (6 layers)**  \n  1. Masked self‑attention (no future look‑ahead).  \n  2. Encoder‑decoder attention (source → target).  \n  3. Feed‑forward network.  \n  4. Residual + layer‑norm.\n\n- **Hyper‑parameters**  \n  - `d_model = 512`, `d_ff = 2048`, `h = 8`, `dropout = 0.1`.

C:\Anaconda3\Lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! reasoning_format is not default parameter.
                    reasoning_format was transferred to model_kwargs.
                    Please confirm that reasoning_format is what you intended.
  warnings.warn(


content='**Key facts & findings**\n\n- **Datasets**  \n  - WMT\u202f2014 EN‑DE: ~4.5\u202fM sentence pairs, BPE\u202f32\u202fk.  \n  - WMT\u202f2014 EN‑FR: ~12\u202fM sentence pairs, BPE\u202f32\u202fk.  \n  - Standard WMT validation/test sets (newstest2013/2014) assumed.\n\n- **Experimental setup**  \n  - 3\u202fM update steps, 4096‑token batches (≈32\u202fk tokens per GPU), 8\u202f×\u202fV100.  \n  - Optimizer & LR schedule not detailed (likely Adam + warm‑up + inverse‑sqrt).  \n  - Inference: beam\u202f4, length penalty\u202f0.6.  \n  - Wall‑clock time reported for 3\u202fM steps.\n\n- **Baselines**  \n  - RNN‑Attention (Bahdanau), ConvS2S (depth‑wise conv), GNMT (8‑layer LSTM), Transformer‑base/big.  \n  - Baseline scores may come from literature; training‑time comparability unclear.\n\n- **Metrics**  \n  - Cased BLEU on newstest2014.  \n  - Training time (hours) on 8\u202f×\u202fV100.\n\n- **Evidence for claims**  \n  - **Faster training**: Transformer‑base 24\u202fh vs' response_

In [300]:
supervisor_result =supervisor_agent_executor.ainvoke({'input':'conduct the reseach on the given research paper', 'context':result_doc_agent['output']})

In [479]:
print(final_output)

**Research‑Coordinator Summary – “Attention Is All You Need” (Vaswani et al., 2017)**  

Below is the full, beginner‑friendly breakdown of the paper, organized into logical components.  Each section draws on the specialist‑agent outputs that were generated with the provided tools (math‑explanation, paper‑overview, and experimental analysis).

---

## 1️⃣ Problem Statement  
- **What the field struggled with:**  
  - Recurrent (RNN) and convolutional (CNN) models process tokens **sequentially**, limiting parallelism and making training slow.  
  - Long‑range dependencies are hard to capture because information must travel step‑by‑step through many layers.  
- **Goal of the paper:** Build a **fully parallel, attention‑only architecture** that can (a) train much faster, and (b) achieve or surpass state‑of‑the‑art machine‑translation quality.

---

## 2️⃣ Proposed Method – The Transformer Architecture  

| Component | Key Idea (bullet‑point) | Why it matters |
|-----------|----------------